# Tuning parameters

This example demonstrates some basics about tuning wake model parameters with `foxes`. All optimizations use the [iwopy](https://github.com/FraunhoferIWES/iwopy) interface in the background (also by Fraunhofer IWES, see link for details). 

In the following we invoke the optimization library [pymoo](https://pymoo.org/) which contains a number of very nice genetic algorithm implementations. Within `foxes` we do that implicitely via the `iwopy` interface.

These are the required imports for this example:

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plotly.offline import iplot
from iwopy.interfaces.pymoo import Optimizer_pymoo

import foxes
import foxes.variables as FV
import foxes.utils.geom2d as gm
# from foxes.opt.problems import TuningProblem
# from foxes.opt.constraints import FarmBoundaryConstraint, MinDistConstraint
# from foxes.opt.objectives import MaxFarmPower

In [5]:
def load_data(df_in,les_files,dDs):
    def mask_turb(y,z,rcy=0,hh=0,rr=63):
        y = np.array(y)
        z = np.array(z)
        ## Make mask of rotor area virtual downstream turbine
        mask = np.zeros((len(z),len(y)))
        for z_idx in range(len(mask)): ## loop over vertical
            z_tmp = z[z_idx] 
            y_left = rcy - np.sqrt(rr**2-(z_tmp-hh)**2) ## use equation of circle
            y_right = rcy + np.sqrt(rr**2-(z_tmp-hh)**2) ## to determine left,right edge 
            for y_idx in range(len(mask[z_idx])): ## loop over horizontal
                y_tmp = y[y_idx]
                if y_left < y_tmp < y_right: ## if between left and right edge
                    mask[z_idx,y_idx] = 1   
        return mask

    import netCDF4 as nc
    
    dDs = np.asarray(dDs)

    u_refs = []
    for i,file in enumerate(les_files):
        nc_file = nc.Dataset(file)
        tmp = []
        for dD in dDs:
            u = nc_file.variables['u'][0,:,:,dD]
            if i == 0:
                y = nc_file.variables['y']
                z = nc_file.variables['z']
                mask = mask_turb(y,z,rcy=1280,hh=90)
            u[mask==0]=np.nan
            tmp += [np.nanmean(u)]
        u_refs += [tmp]
    mask = np.asarray(mask)
    u_refs = np.asarray(u_refs)
    y = np.asarray(y)
    z = np.asarray(z)
    return y,z,u_refs,mask

path_local = '/home/sengers/data/foxes-cases/LES_ref/'
df_in = pd.read_csv(path_local+'LES_in.csv')
les_files = [path_local+k+'.nc' for k in ['LES_yaw00','LES_yaw30','LES_yaw-30']]
y,z,u_refs,mask = load_data(df_in,les_files,[6])

u_refs

/tmp/ipykernel_528/1397851891.py:9: RuntimeWarning:

invalid value encountered in sqrt

/tmp/ipykernel_528/1397851891.py:10: RuntimeWarning:

invalid value encountered in sqrt



array([[6.16247756],
       [6.9719514 ],
       [7.21692477]])

In [7]:
def def_points(farm_results, xs, y, z):
    import foxes.constants as FC
    from foxes.utils import wd2wdvec, wd2uv
    # prepare:
    n_x = np.append(wd2wdvec(270-180), [0.0], axis=0) ## -180 to get [1,0,0]
    n_z = np.array([0.0, 0.0, 1.0])
    n_y = np.cross(n_z, n_x)

    # project to axes:
    # xyz = np.zeros((n_states, n_turbines, 3), dtype=FC.DTYPE)
    xyz = np.zeros((1,1,3), dtype=FC.DTYPE) ## (n_states, n_turbines, n_dimensions)
    xyz[:, :, 0] = farm_results[FV.X]
    xyz[:, :, 1] = farm_results[FV.Y]
    xyz[:, :, 2] = farm_results[FV.H]
    xx = np.einsum("std,d->st", xyz, n_x)
    yy = np.einsum("std,d->st", xyz, n_y)
    zz = np.einsum("std,d->st", xyz, n_z)
    del xyz

    y_pos, y_res = np.linspace(
        np.min(y),
        np.max(y),
        num=int((np.max(y) - np.min(y)) / 5) + 1,
        endpoint=True,
        retstep=True,
        dtype=None,
    )
    z_pos, z_res = np.linspace(
        np.min(z),
        np.max(z),
        num=int((np.max(z) - np.min(z)) / 5) + 1,
        endpoint=True,
        retstep=True,
        dtype=None,
    )
    
    N_y, N_z = len(y_pos), len(z_pos)
    n_pts = len(y_pos) * len(z_pos)
    data_all = []
    for x in xs:
        x_pos = x
        g_pts = np.zeros((1, N_y, N_z, 3), dtype=FC.DTYPE)
        g_pts[:] += x_pos * n_x[None, None, None, :]
        g_pts[:] += y_pos[None, :, None, None] * n_y[None, None, None, :]
        g_pts[:] += z_pos[None, None, :, None] * n_z[None, None, None, :]
        g_pts = g_pts.reshape(1, n_pts, 3)

    return np.asarray(g_pts)

def_points(farm_results, [750], np.arange(-100,101,5), z)

In [ ]:
def mask_turb(y,z,rcy=0,hh=0,rr=63):
    y = np.array(y)
    z = np.array(z)
    ## Make mask of rotor area virtual downstream turbine
    mask = np.zeros((len(z),len(y)))
    for z_idx in range(len(mask)): ## loop over vertical
        z_tmp = z[z_idx] 
        y_left = rcy - np.sqrt(rr**2-(z_tmp-hh)**2) ## use equation of circle
        y_right = rcy + np.sqrt(rr**2-(z_tmp-hh)**2) ## to determine left,right edge 
        for y_idx in range(len(mask[z_idx])): ## loop over horizontal
            y_tmp = y[y_idx]
            if y_left < y_tmp < y_right: ## if between left and right edge
                mask[z_idx,y_idx] = 1   
    return mask

We now setup the model book and a wind farm with 1 turbine with the correct yaw angle, with some initial values for the tuning paramters

In [6]:
## Initialize
df_in.index = range(len(df_in))
yawm = [[k] for k in df_in['yawm']]


states = foxes.input.states.StatesTable(
    data_source=df_in, ## df
    output_vars=[FV.WS, FV.WD, FV.SHEAR, FV.RHO, FV.TI],
    var2col={FV.WS: "WS", FV.SHEAR: "shear", FV.TI: "TI"},
    fixed_vars={FV.RHO: 1.225, FV.H: 90.0, FV.WD: 270.0},
    profiles={FV.WS: "ShearedProfile"},
)

## Build and execute
mbook = foxes.ModelBook()
mbook.turbine_types["NREL5"] = foxes.models.turbine_types.PCtFile(
    "NREL-5MW-D126-H90.csv"
)
mbook.turbine_models["set_yawm"] = foxes.models.turbine_models.SetFarmVars()
mbook.turbine_models["set_yawm"].add_var(FV.YAWM, yawm)

farm = foxes.WindFarm()
farm.add_turbine(
    foxes.Turbine(
        xy=[0.0, 0.0],
        turbine_models=["set_yawm", "yawm2yaw", "NREL5", "kTI_05"],
        # turbine_models=["tuning_opt","set_yawm", "yawm2yaw", "NREL5"],
    )
)

algo = foxes.algorithms.Downwind(
    mbook,
    farm,
    states,
    rotor_model="centre",
    wake_models=["PorteAgel_linear", "CrespoHernandez_max"],
    wake_frame="yawed",
    partial_wakes_model="auto",
    chunks=None,
    verbosity=0,
)
farm_results = algo.calc_farm()

# problem = TuningProblem("tuning_opt", algo,vars_names=[FV.PA_ALPHA,FV.PA_BETA], vars_init=[0.58,0.07])
# problem.initialize()

Turbine 0, T0: set_yawm, yawm2yaw, NREL5, kTI_05


Notice the appearing wake frame model `yawed_tuning`. This is not part of the model book but will be defined shortly by the optimization problem. 

Let's new define the optimization problem. Our objective is the minimazation of the total wind farm power:

In [ ]:
# problem = TuningProblem("tuning", algo)
# problem.add_objective(MinCostPower(problem))
# problem.initialize()

Notice that the two added constraint models imply a total of 55 individual constraint component functions.

Next, we setup the optimizer. In our case we use the genetic algorithm [GA from pymoo](https://pymoo.org/algorithms/soo/ga.html) via the [iwopy](https://github.com/FraunhoferIWES/iwopy) interface, here in vectorized form (flag `vectorize=True`), with 100 generations (`n_max_gen=100`) with population size 50 (`pop_size=50`):

In [ ]:
# solver = Optimizer_pymoo(
#     problem,
#     problem_pars=dict(vectorize=True),
#     algo_pars=dict(
#         type="GA", 
#         pop_size=50, 
#         seed=42,
#     ),
#     setup_pars=dict(),
#     term_pars=dict(
#         type="default",
#         n_max_gen=100,
#         ftol=1e-6,
#         xtol=1e-3,
#     ),
# )
# solver.initialize()
# solver.print_info()

After all the setup we can now solve the problem:

In [ ]:
# results = solver.solve()
# solver.finalize(results)

# print()
# print(results)
# print(results.problem_results)